# preprocess Cell Track Orange Validation Data triplett

* bright field (bf)
* fluorescence (fluor)
* lensfree

In [ ]:
import sys
import os
import re
import csv
import cv2
import numpy as np

import subprocess

from preprocessmicroscopydata import ImagePreProcessor

import codecs

import matplotlib.pyplot as plt
%matplotlib inline

def getFiles(fpath):
    files = [f for f in os.listdir(fpath) if os.path.isfile(os.path.join(fpath,f)) and re.match(r'.*\.*(tif|png)$',f)]
    files.sort(key=lambda f: f.split()[0])
    return files

In [ ]:
basedir = '[DIRECTORY_OF_DATA]'

bf_imgs = getFiles(os.path.join(basedir,'bf'))
fluo_imgs = getFiles(os.path.join(basedir,'fluor'))
lm_imgs = getFiles(os.path.join(basedir,'lensfree_unaligned_raw'))

In [ ]:
# preprocess microscopy images (bf and fluor)

ImgProc_micro = ImagePreProcessor()
ImgProc_micro.claheClip = 6
ImgProc_micro.claheTileSize = 64
ImgProc_micro.NLM1=50
ImgProc_micro.NLM2=20
ImgProc_micro.NLM3=21

img_fluo = cv2.imread(os.path.join(basedir,'bf',fluo_imgs[0]),0)
img_bf = cv2.imread(os.path.join(basedir,'fluor',bf_imgs[0]),0)
    
img_bf_proc = ImgProc_micro.histogramEq((img_bf))
img_fluo_proc2 = ImgProc_micro.histogramEq(ImgProc_micro.subtractBackground(img_fluo))

img_bf_den = ImgProc_micro.fastNonLocalDenoising(img_bf_proc)
img_fluo_den = ImgProc_micro.fastNonLocalDenoising(img_fluo_proc2)


cv2.imwrite(os.path.join(basedir,'bf_proc',bfimg),img_bf_den)
cv2.imwrite(os.path.join(basedir,'fluor_proc',fluo_imgs[idx]),img_fluo_den)

In [ ]:
# preprocess raw LED images

ImgProc = ImagePreProcessor()
ImgProc.claheClip = 6
ImgProc.claheTileSize = 12

img_raw = cv2.imread(os.path.join(basedir,'lensfree_unaligned_raw',lm_imgs[0]),1)
img = cv2.cvtColor(img_raw, cv2.COLOR_BGR2GRAY)
img_proc = ImgProc.histogramEq(ImgProc.subtractBackground(img))